# Chapter 4. Improvement of Word2vec

## 1. Embedding 계층 - (입력-은닉층) 순전파 및 역전파의 불필요한 계산을 생략.

만약 단어의 수가 너무 많을 경우 (i.e. 100만 개 -> Input layer: (1,100만))
Input * Weight 의 행렬곱 연산량이 너무 많다. i.e. (1,100만) * (100만,500)

특정 단어가 입력될 경우 (ID = 0, 1, 2, ..., 100만 에서 j번째 단어라고 가정), <br> 
행렬곱의 연산 결과는 Weight matarix의 j번째 행이 될 것이다.

따라서, 이 계산을 간략화해줄 Embedding 계층을 넣어, 불필요한 계산을 생략하도록 하자.

In [3]:
# Numpy example : 특정 행 뽑아내기
import numpy as np

A = np.arange(21).reshape(7,3)

print(A)
# [idx=0] : 1번째 행 뽑아 내기
A[0]

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]


array([0, 1, 2])

In [4]:
# idx=4 : 5번째 행
A[4]

array([12, 13, 14])

In [8]:
# 특정 행만 뽑아내기 : 3번째, 7번째 행
row_idx = np.array([2,6])
A[row_idx]
# 아래 표현과 동일
# A[[2,6]]
#
# numpy 배열과 일반 배열 모두 작동함!

array([[ 6,  7,  8],
       [18, 19, 20]])

In [9]:
# Embedding 계층 구현

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout) # 중복문제 해결을 위해 해당 행에 할당이 아닌 더하기. dout를 dW의 idx번째 행에 더해줌
        # 아래와 동일한 연산을 수행함.
        # for i, word_id in enumerate(self.idx):
        #   dW[word_id] += dout[i]
        # 일반적으로 파이썬에서 for문 보다는 넘파이의 내장 메서드를 사용하는 편이 더 빠름. 
        
        return None

위의 Embedding 계층이 기존 Word2vec의 matmul 계층을 대체한다. <br> 
(사실 하는 역할 및 연산은 똑같지만, 불필요한 계산을 줄인 것. 메모리 절약 효과도 있다!)

## 2. Softmax 계산 간략화 방법론

### (1) Negative sampling

- (은닉층-출력층) 연산 또한 계산량이 많음.
- softmax의 경우도 연산량이 많음. (exp 연산을 N번 시행, N=단어 개수)

#### (i). 이진 분류로 근사.
    - 출력층 뉴런은 1개만 있고, 이진 분류로 가정. (이 단어냐, 아니냐? Yes or no)
    - Sigmoid 함수로 확률변환, cross-entropy로 cost function 설정.
    
- - -

    - 특정 단어에 대해 1이 정답이라고 가정하면 해당 뉴런 외의 나머지 뉴런에 대해서는 0에 가깝게 나오는 것을 원함.
    - 적은 수의 부정적 예를 샘플링해 사용 (?)
    - 정답인 예에 대해 학습하고, 부정적 예 (정답=0) 몇 개 샘플링 (선별)하여 각각의 손실함수의 합을 최종 손실로 한다.

#### (ii). 샘플링 방법
    - 말뭉치의 단어 빈도를 기준으로 샘플링.
    - 많은 빈도수에 대해 더 많이 나오게, 희소할 수록 적게 나오게.

In [10]:
# 샘플링 예시 : np.random.choice로 배열 원소 중 랜덤하게 샘플링.
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'you'

In [11]:
# 랜덤하게 5개 샘플링 (중복 가능)
np.random.choice(words,size=5)

array(['goodbye', 'hello', 'hello', 'I', 'I'], dtype='<U7')

In [12]:
# 랜덤하게 5개 샘플링 (중복 불가)
np.random.choice(words,size=5,replace=False)

array(['goodbye', 'I', 'you', '.', 'say'], dtype='<U7')

In [15]:
# 확률 분포에 따른 샘플링
# np.sum(p) == 1 이어야 함!
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
#print(np.sum(p))
np.random.choice(words, p=p)

1.0000000000000002


'you'

* word2vec 네거티브 샘플링에서는 앞의 확률분포에서 한 가지를 수정하라고 권고 => 기본 확률분포에 0.75 제곱
    * 출연 확률이 낮은 단어를 버리지 않기 위해서.
    * 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있음.
    * 0.75에 이론적 의미는 없으니 다른 값으로 설정해도 됨.
    
#### Unigram, bigram, trigram, ...
    - 하나의 (연속된) 단어. 바이그램은 2개, 트라이그램은 3개
    - 한 단어를 대상으로 확률분포를 만든다.
    - 바이그램 버전이면, 두 단어로 구성된 대상에 대한 확률분포를 만든다.

In [29]:
# 유용할 수 있는 numpy method: np.c_
# np.c_ : 가로 방향으로 배열을 붙인다.
# 단, 같은 크기의 1차원 array 여러개를 붙일 경우, 각 1차원 배열을 열로 간주하고 {(N,1) 행렬} 세로로 붙임.
A = np.array([1,2,3])
B = np.array([4,5,6])
C = np.array([7,8,9])
print(np.c_[A,B,C])
print(np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])])
print(np.c_[np.array([[1,2],[3,4]]),np.array([[5,6],[7,8]])])
# 1차원 배열 (3,) => (3,1)
print(np.c_[[1,2,3]])

[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 2 3 0 0 4 5 6]]
[[1 2 5 6]
 [3 4 7 8]]
[[1]
 [2]
 [3]]
